Modèle ARIMA avec Python – Prévisions de séries temporelles
===


Le modèle ARIMA (moyenne mobile intégrée autorégressive) avec Python donne la possibilité de faire des prévisions basées sur des observations historiques.

Une série temporelle peut être décomposée en 3 composantes:
- Tendance : Mouvement de hausse et de baisse des données au cours du temps sur une longue période (par exemple le prix de l’immobilier)
- Saisonnalité : Variance saisonnière (par exemple une augmentation de la demande de glace pendant l’été)
- Bruit : Des pics et des creux à intervalles aléatoires

Avant d’appliquer un modèle statistique sur une série temporelle, nous devons nous assurer qu’elle est stationnaire.
**La moyenne (hauteur moyenne des vagues), la variance (taille des vagues) et la covariance (écart entre vague) de la série ne doivent pas être fonction du temps !**

In [1]:
from tools import find_file

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

ImportError: cannot import name 'DtypeArg' from 'pandas._typing' (C:\Anaconda3\lib\site-packages\pandas\_typing.py)

In [ ]:
df = pd.read_csv(find_file('airline_passengers.csv'), parse_dates = ['Month'], index_col = ['Month'])

df.head()

In [ ]:
plt.xlabel('Date')
plt.ylabel('Nombre de passagers aériens')
plt.plot(df)